In [ ]:
import uuid

In [ ]:
user = 'olferuk'
password = '123'
salt = '-=-=-'


user2 = 'mary'
password2 = '234'


In [ ]:
hash(user + password + salt)

In [ ]:
hash(user2 + password2 + salt)

----

In [ ]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

from tqdm.auto import  tqdm

from urllib.parse import urljoin

import re

In [ ]:
root = 'http://books.toscrape.com/'

In [ ]:
r = requests.get(root)

In [ ]:
r.status_code, r.ok

In [ ]:
soup = BeautifulSoup(r.text)

In [ ]:
def parse_category(x):
    return {
        'link': f"http://books.toscrape.com/{x.select_one('a')['href']}",
        'name': x.select_one('a').text.strip(),
    }

In [ ]:
categories = pd.DataFrame([parse_category(x) for x in soup.select('ul.nav li')[1:]])

In [ ]:
decode = {
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5
}

data = []
for i in tqdm(categories.index):
    
    theme = categories.loc[i, 'name']
    url = categories.loc[i, 'link']
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    
    articles = soup.select('article.product_pod')
    
    for article in articles:
        title = article.select_one('h3 a').text.strip()
        price = article.select_one('p.price_color').text.strip()
        is_available = 'In stock' in article.select_one('p.availability').text
        stars = article.select_one('p.star-rating')
        rating = decode[[x for x in stars['class'] if x != 'star-rating'][0]]
        img = urljoin(url, article.select_one('img')['src'])
        
        data.append({
            'title': title,
            'price': price,
            'is_available': is_available,
            'rating': rating,
            'img': img,
            'theme': theme,
        })

df = pd.DataFrame(data)

df.price = df.price.apply(lambda p: float(re.findall(r'[\d\.]+', p)[0]))

In [ ]:
df